In [1]:
import cfgrib

import xarray as xr
import numpy as np
import multiprocessing as mp

from glob import glob

import warnings
warnings.filterwarnings('ignore')

In [2]:
archive_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/archive/'

In [3]:
variable_filters = [{'typeOfLevel': 'isobaricInhPa'},
           {'typeOfLevel': 'unknown', 'stepType':'instant'},
           {'typeOfLevel': 'surface'}]

filter_keys = [['gh', 't', 'r', 'u', 'v'],
              ['pwat'],
              ['orog', 'tp']]

def open_grib(grib_file):
    
    grib_data = []
    
    for variable_filter, keep_keys in zip(variable_filters, filter_keys):
        try:
            grib_data.append(xr.open_dataset(grib_file, engine='cfgrib', 
                            backend_kwargs={'errors':'ignore', 
                                            'filter_by_keys':variable_filter}
                                            )[keep_keys])

        except:
            pass
        
    try:
        return xr.merge(grib_data
                           ).drop(['time', 'level', 'step', 'surface']
                           ).rename({'isobaricInhPa':'level', 'valid_time':'time'})

    except:
        return None

In [ ]:
init_hour_files = []

for date_dir in sorted(glob(archive_dir + '*')):
    
    date = date_dir.split('/')[-1]
    
    for init_hour in np.arange(0, 12.1, 12):
        
        for fhr in np.arange(3, 12.1, 3):
        
            try:
                init_hour_files.append(
                    glob(date_dir + '/models/gfs0p25/*%s%02d*f%03d*.WE.grib2'%(
                        date, init_hour, fhr)))
            except:
                pass
                    
init_hour_files = np.hstack(init_hour_files)

In [ ]:
len(init_hour_files)

In [ ]:
with mp.get_context('fork').Pool(128) as p:
    returned_grib_data = p.map(open_grib, init_hour_files, chunksize=None)
    p.close()
    p.join()

In [ ]:
returned_grib_data = [r for r in returned_grib_data if r is not None]

In [ ]:
data = xr.concat(returned_grib_data, dim='time')

In [ ]:
data.to_netcdf('/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/gfs_aggregate.nc')

In [5]:
data = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/gfs_aggregate.nc')
data

<xarray.Dataset>
Dimensions:    (latitude: 81, level: 31, longitude: 121, time: 8879)
Coordinates:
  * level      (level) int64 1 2 3 5 7 10 20 30 ... 800 850 900 925 950 975 1000
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 230.0 230.2 230.5 230.8 ... 259.5 259.8 260.0
  * time       (time) datetime64[ns] 2015-01-15T03:00:00 ... 2019-06-01
Data variables:
    gh         (time, level, latitude, longitude) float32 ...
    t          (time, level, latitude, longitude) float32 ...
    r          (time, level, latitude, longitude) float32 ...
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
    pwat       (time, latitude, longitude) float32 ...
    orog       (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...